In [1]:
import gzip,os,tarfile,sys
sys.path.append(os.pardir+'/src')
from settings import *
import datetime
import traceback
import logging
import pandas as pd
from pandas import DataFrame
import xml.etree.ElementTree as et
%matplotlib inline 

↓myCsvは以下の関数で作成

In [2]:
# ナカシマのcsvの処理。bodyが入っていて、日本語しか扱っていない。csv2に格納
def my_convertToCSV(file_name):
    start = datetime.datetime.now()

    # 引数として渡されたtar.gzファイルを解凍し、オープン
    f = gzip.open(os.path.join(DATA_DIR,'EID42186/original_Data',file_name), 'r')

    # CSVファイルのカラムに対応する配列を初期化
    ids = []
    headlines = []
    timeofarrivals = []
    bodys = []
    #langs = []

    # xmlを解析し、rootを取得
    tree = et.parse(f)
    elem = tree.getroot()
    # エラーとなった記事をカウントする変数を初期化
    fail_cnt = 0

    # 解析したxmlから、記事単位で要素を取得
    contents = elem.getiterator('ContentT')

    for content in contents:
        try:
            # 記事の言語情報を取得し、日本語か英語の記事であれば以降の処理を実施
            lang = content.find(".//LanguageString").text
            #if lang == 'JAPANESE' or lang == 'ENGLISH':
            if lang == 'JAPANESE':
                if content.find(".//Body").text is not ' ':
                    # 言語、ID、タイトル、タイムスタンプを配列に格納
                    #langs.append(lang)
                    ids.append(content.find(".//Id/SUID").text)
                    headlines.append(content.find(".//Headline").text)
                    timeofarrivals.append(content.find(".//TimeOfArrival").text)
                    bodys.append(content.find(".//Body").text)
        except:
            # 読み取りに失敗した場合はカウント
            fail_cnt += 1
            #traceback.print_exc()
    
    print('fail_cnt:', fail_cnt)
    
    # カラムに対応する配列を用いてDataFrameを作成
    df = DataFrame({"Id":ids})
    df['Headline']=headlines
    df['Body']=bodys
    df['TimeOfArrival']=timeofarrivals
    df = df.drop_duplicates(['Id'])
    # DataFrameをCSVファイルとして保存
    df.to_csv(os.path.join(DATA_DIR,'EID42186/myCsv',file_name.replace(".xml.gz",".csv")),encoding='utf8',index=False)
    f.close()

    print('file_name:' + file_name + ' time:' + str(datetime.datetime.now()-start) + ' record_count:' + str(len(df)) + ' fail_count:' + str(fail_cnt))

In [3]:
#　全ファイル取得
files_2014 = os.listdir(DATA_DIR+'/EID42186/myCsv/2014')
files_2015 = os.listdir(DATA_DIR+'/EID42186/myCsv/2015')
files_2016 = os.listdir(DATA_DIR+'/EID42186/myCsv/2016')
# headLineのkey
headline_key = pd.read_csv(DATA_DIR+'/EID42186/headline_key.csv',header=None).unstack().tolist()

In [4]:
len(headline_key)

26306

In [5]:
df = pd.DataFrame(index=[], columns=['Id', 'Headline','Body', 'TimeOfArrival'])

In [6]:
for file in files_2014:
    csv = pd.read_csv(os.path.join(DATA_DIR,'EID42186/myCsv/2014',file.replace(".xml.gz",".csv")))
    df = pd.concat([df, csv], ignore_index=True)
for file in files_2015:
    csv = pd.read_csv(os.path.join(DATA_DIR,'EID42186/myCsv/2015',file.replace(".xml.gz",".csv")))
    df = pd.concat([df, csv], ignore_index=True)
for file in files_2016:
    csv = pd.read_csv(os.path.join(DATA_DIR,'EID42186/myCsv/2016',file.replace(".xml.gz",".csv")))
    df = pd.concat([df, csv], ignore_index=True)

df = df.drop_duplicates(['Body'])

In [7]:
#csv = pd.read_csv(os.path.join(DATA_DIR,'EID42186/myCsv/2014',files_2014[1].replace(".xml.gz",".csv")))
#csv.head()
#csv.Headline in headline_key

In [8]:
df = df[df['Headline'].map(lambda x: x in headline_key)]
#df['Headline'] = df['Headline'].apply(lambda x: x.split('：')[0].replace('*',''))
#df = df.ix[:,['Headline','Body','Id', 'TimeOfArrival']]
#df.columns = ['remarker', 'remark(Body)','Id', 'TimeOfArrival']
#df.head()

In [9]:
df.shape

(12351, 4)

↑headline_keyより少ない？？

In [10]:
df.head()

,Id,Headline,Body,TimeOfArrival
2,MYP7ON6JIJUO,日銀：緩和２年限定せず、黒田総裁「物価２％」安定まで－読売,\n【記者：黄恂恂】\n １月１日（ブルームバーグ）： 読売新聞が黒田東彦総裁のイン\n...,2014-01-01T01:19:35.253+00:00
3,MYPCHB6TTDSP,日銀総裁：日本経済「堅実に成長」、消費増税の影響少ない－共同,\n【記者：黄恂恂】\n １月１日（ブルームバーグ）：共同通信が黒田東彦日銀総裁インタ\n...,2014-01-01T03:03:12.005+00:00
5,MYQZ2W6K511O,安倍首相：ＯＥＣＤ閣僚理事会に出席、仏大統領と会談へ－朝日新聞,\n【記者：沢和世】\n １月２日（ブルームバーグ）： 情報源は示していない。\n<ul...,2014-01-02T00:08:56.492+00:00
33,MYR41E6TTDS6,中国：昨年12月のＨＳＢＣ製造業ＰＭＩ改定値、50.5－前月から低下,\n【Bloomberg News】\n １月２日（ブルームバーグ）： 英ＨＳＢＣホールデ...,2014-01-02T01:56:02.272+00:00
34,MYNNZQ6TTDS5,中国人民銀：「適切な水準」の流動性を維持へ－四半期毎の金融政策委,\n【Bloomberg News】\n 12月31日（ブルームバーグ）： 四半期ごとに...,2013-12-31T05:16:38.480+00:00


In [11]:
df.to_csv('../data/EID42186/nak_task2_data.csv',encoding='utf8',header=True,index=False)